In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
#Anlaufdaten importieren
calls = pd.read_excel(r'calls.xlsx')

#Irrelevante Features entfernen
calls = calls[calls.Schiffstyp != 'Kahn']
calls = calls.rename(columns={'Ankunft(soll)':'ETA','Ankunft':'ATA','Abfahrt(soll)':'ETD', 'Abfahrt':'ATD'})
calls = calls.drop(['Importreise','Exportreise','Schiffstyp'],axis=1)

#Lösche alle Reihen ohne Löschdaten
calls = calls[calls.Löschbeginn != 'no information available']
calls.dropna(subset=['Löschbeginn','Löschende','ATA'],inplace= True)

#Zeitangaben in datetime umwandeln
calls[['ETA','ATA','Löschbeginn','Löschende','Ladebeginn','Ladeende','ETD','ATD']]=calls[['ETA','ATA','Löschbeginn','Löschende','Ladebeginn','Ladeende','ETD','ATD']].apply(pd.to_datetime, dayfirst=True)

In [3]:
#Flottendaten importieren
fleet = pd.read_excel(r'fleet.xlsx')

#Flotte aufräumen
fleet = fleet.rename(columns={'CALL_SIGN':'Funkcode'})
fleet = fleet.drop(['DWT','BAUJAHR','SHIPNAME','ImoNr'], axis=1)

In [4]:
#Klimadaten (Tageswerte) importieren und bereinigen
winddata = pd.read_excel(r'weather.xlsx', sheet_name="wind")
winddata = winddata.drop(['STATIONS_ID', 'eor', 'QN_3'], axis=1)
raindata = pd.read_excel(r'weather.xlsx', sheet_name="rain")
raindata = raindata.drop(['STATIONS_ID', 'eor', 'QN_8'], axis=1)
temperaturedata = pd.read_excel(r'weather.xlsx', sheet_name="temperature")
temperaturedata = temperaturedata.drop(['STATIONS_ID', 'eor', 'QN_9'], axis=1)
visibilitydata = pd.read_excel(r'weather.xlsx', sheet_name="visibility")
visibilitydata = visibilitydata.drop(['STATIONS_ID', 'eor', 'V_VV_I', 'QN_8'], axis=1)

#Klimadaten mergen
weather = pd.merge(winddata, raindata, on='MESS_DATUM')
weather = pd.merge(weather, temperaturedata, on='MESS_DATUM')
weather = pd.merge(weather, visibilitydata, on='MESS_DATUM')
weather['MESS_DATUM']= weather['MESS_DATUM'].apply(pd.to_datetime, format="%Y%m%d%H")
weather= weather.set_index('MESS_DATUM')

In [5]:
#Anlaufdaten und Flottendaten mergen
total = pd.merge(calls, fleet, on='Funkcode')

#Klassifikation der Anläufe ermitteln
def vessel_classification(data):
    if data <= 1499:
        return('Small')
    elif (data >= 1500) & (data < 4999): 
        return('Medium')
    elif (data >= 5000) & (data < 10999): 
        return('Large')
    elif (data >= 11000) & (data < 17999): 
        return('VeryLarge')
    else: 
        return('UltraLarge')
classifier = list(map(vessel_classification, total['TEU']))
total.insert(11, value=classifier, column="Vessel_Class")

In [6]:
#Wetterbeobachtungen für jeden Anlauf ermitteln
def mean_temp(data1, data2):
    slice = weather.truncate(before=data1, after=data2)
    mean = slice['   F'].mean()
    return mean

def mean_rainfall(data1, data2):
    slice = weather.truncate(before=data1, after=data2)
    mean = slice['  R1'].mean()
    return mean

def mean_temperature(data1, data2):
    slice = weather.truncate(before=data1, after=data2)
    mean = slice['TT_TU'].mean()
    return mean

def mean_visibility(data1, data2):
    slice = weather.truncate(before=data1, after=data2)
    mean = slice['V_VV'].mean()
    return mean

windspeed = list(map(mean_temp, total['ATA'], total['ATD']))
total.insert(15, value=windspeed, column="AVG_WINDSPEED")

rainfall = list(map(mean_rainfall, total['ATA'], total['ATD']))
total.insert(16, value=rainfall, column="AVG_RAINFALL")

temperature = list(map(mean_temperature, total['ATA'], total['ATD']))
total.insert(17, value=temperature, column="AVG_TEMPERATURE")

visibility = list(map(mean_visibility, total['ATA'], total['ATD']))
total.insert(18, value=temperature, column="AVG_VISIBILITY")

In [7]:
#Löschdauer berechnen
def timedelta_minutes(value):
    days, seconds = value.days, value.seconds
    minutes = (days * 1440) + (seconds/ 60)
    return int(minutes)
total['Unloading']=(total['Löschende']-total['Löschbeginn'])
total['UNLOADING_mt']= total['Unloading'].apply(timedelta_minutes)
total = total.drop(['Unloading'],axis=1)

#Lösche negative Löschdauern durch falsche Eingaben
total.drop(total[total.UNLOADING_mt <= 0].index, axis=0, inplace=True)

#Vorbereitungszeit berechnen
total['setup']=(total['Löschbeginn']-total['ATA'])
total['SETUP_mt']= total['setup'].apply(timedelta_minutes)
total = total.drop(['setup'],axis=1)

#Setze negative Vorbereitungszeiten auf null
total['SETUP_mt'] = total.SETUP_mt.clip(lower=0)

#Zeit von "angelegt bis Ende Entladen" berechnen
total['UNLOADING_TOTAL'] = total['SETUP_mt']+total['UNLOADING_mt']

#Zeit von "Entladeende bis Abfahrt" berechnen
total['UNLOAD_TO_DEP']=(total['ATD']-total['Löschende'])
total['UNLOAD_TO_DEP_mt']= total['UNLOAD_TO_DEP'].apply(timedelta_minutes)
total = total.drop(['UNLOAD_TO_DEP'],axis=1)


In [8]:
#Gesamtliste speichern
total.to_excel(r'masterlist_total.xlsx')

In [9]:
#Extrakt für Vorhersagemodelle speichern
extrakt = total[['Terminal','TEU', 'Länge','Tiefgang','Breite','AVG_WINDSPEED', 'AVG_RAINFALL', 'AVG_VISIBILITY','AVG_TEMPERATURE', 'UNLOADING_TOTAL', 'SETUP_mt', 'UNLOADING_mt']]
extrakt.to_excel(r'..\Vorhersagemodelle\masterlist_total_extrakt.xlsx')